In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH RECURSIVE input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            hno,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[1], ', ')[1] AS BIGINT) px,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[1], ', ')[2] AS BIGINT) py,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[1], ', ')[3] AS BIGINT) pz,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[2], ', ')[1] AS BIGINT) vx,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[2], ', ')[2] AS BIGINT) vy,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[2], ', ')[3] AS BIGINT) vz
        FROM (
            SELECT
                GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) hno,
                UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    ), cross_2d AS (
        -- pf = p + v*t
        -- t1.px + t1.vx * ta = t2.px + t2.vx * tb
        -- t1.py + t1.vy * ta = t2.py + t2.vy * tb
        
        -- (t1.py - t2.py + t1.vy * ta) / t2.vy = tb
        -- t1.px + t1.vx * ta = t2.px + t2.vx * ((t1.py - t2.py + t1.vy * ta) / t2.vy)
        -- t1.px + t1.vx * ta = t2.px + (t2.vx / t2.vy) * (t1.py - t2.py + t1.vy * ta)
        -- t1.px + t1.vx * ta = t2.px + (t2.vx / t2.vy) * (t1.py - t2.py) + (t2.vx / t2.vy) * (t1.vy * ta)
        -- t1.px - t2.px - (t2.vx / t2.vy) * (t1.py - t2.py) = (t2.vx / t2.vy) * (t1.vy * ta) - t1.vx * ta
        -- t1.px - t2.px - (t2.vx / t2.vy) * (t1.py - t2.py) = (t2.vx * t1.vy / t2.vy - t1.vx) * ta
        -- (t1.px - t2.px - (t2.vx / t2.vy) * (t1.py - t2.py)) / (t2.vx * t1.vy / t2.vy - t1.vx) =  ta
        
        SELECT hno_a, hno_b, ta, tb, cross_x, cross_y
        FROM (
            SELECT
                t1.hno hno_a,
                t2.hno hno_b,
                (t1.px - t2.px - (t2.vx / t2.vy) * (t1.py - t2.py)) / (t2.vx * t1.vy / t2.vy - t1.vx) ta,
                (t1.py - t2.py + t1.vy * ta) / t2.vy tb,
                t1.px + t1.vx * ta cross_x,
                t1.py + t1.vy * ta cross_y
            FROM t t1, t t2
            WHERE t1.hno > t2.hno
        )
        WHERE cross_x BETWEEN 200000000000000 AND 400000000000000
        AND cross_y BETWEEN 200000000000000 AND 400000000000000
        AND ta >= 0
        AND tb >= 0
    )
    SELECT COUNT(*) ans
    FROM cross_2d
""" % input)

┌───────┐
│  ans  │
│ int64 │
├───────┤
│ 24627 │
└───────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            hno,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[1], ', ')[1] AS BIGINT) px,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[1], ', ')[2] AS BIGINT) py,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[1], ', ')[3] AS BIGINT) pz,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[2], ', ')[1] AS BIGINT) vx,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[2], ', ')[2] AS BIGINT) vy,
            CAST(STRING_SPLIT(STRING_SPLIT(i, ' @ ')[2], ', ')[3] AS BIGINT) vz
        FROM (
            SELECT
                GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) hno,
                UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    ), 
        -- pf = p + v*t

        -- px0 + vx0 * t1 - px1 - vx1 * t1 = 0
        -- py0 + vy0 * t1 - py1 - vy1 * t1 = 0
        -- pz0 + vz0 * t1 - pz1 - vz1 * t1 = 0

        -- px0 + vx0 * t2 - px2 - vx2 * t2 = 0
        -- py0 + vy0 * t2 - py2 - vy2 * t2 = 0
        -- pz0 + vz0 * t2 - pz2 - vz2 * t2 = 0
        
        -- px0 + vx0 * t3 - px3 - vx3 * t3 = 0
        -- py0 + vy0 * t3 - py3 - vy3 * t3 = 0
        -- pz0 + vz0 * t3 - pz3 - vz3 * t3 = 0

        -- unfortunally this is only solvable with specific tools

        -- thanks to https://www.reddit.com/r/adventofcode/comments/18pnycy/comment/kicuapd/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
        -- it is possible to simplify it by choosing points with the same velocity in at least one axis and changing the referential coordinates

    relative_axis AS (
        SELECT p1, p2, p3, ARRAY_AGG(p2r1) p2r1, ARRAY_AGG(p3r1) p3r1
        FROM (
            SELECT p1, p2, p3,
                UNNEST(p2) - UNNEST(p1) p2r1,
                UNNEST(p3) - UNNEST(p1) p3r1
            FROM (
                SELECT
                    ARRAY[p1.px, p1.py, p1.pz, p1.vx, p1.vy, p1.vz] p1,
                    ARRAY[p2.px, p2.py, p2.pz, p2.vx, p2.vy, p2.vz] p2,
                    ARRAY[p3.px, p3.py, p3.pz, p3.vx, p3.vy, p3.vz] p3,
                FROM (
                    SELECT
                        t_array[1] p1,
                        t_array[2] p2,
                        t_array[3] p3
                    FROM (
                        SELECT vx, ARRAY_AGG(t) t_array
                        FROM t
                        GROUP BY vx
                        HAVING LENGTH(ARRAY_AGG(t)) >= 3
                        LIMIT 1
                    )
                )
            )
        )
        GROUP BY p1, p2, p3
    ), cross_all AS (
        -- cpx2r1 = px2r1 + vx2r1 * ct2 = px2r1 -- vx2r1 = 0
        -- cpy2r1 = py2r1 + vy2r1 * ct2
        -- cpz2r1 = pz2r1 + vz2r1 * ct2

        -- cpx3r1 = px3r1 + vx3r1 * ct3 = px3r1 -- vx3r1 = 0
        -- cpy3r1 = py3r1 + vy3r1 * ct3
        -- cpz3r1 = pz3r1 + vz3r1 * ct3

        -- since the angle of colision is the same
        -- cpy2r1 / cpx2r1 = cpy3r1 / cpx3r1
        -- cpz2r1 / cpx2r1 = cpz3r1 / cpx3r1

        -- ( py2r1 + vy2r1 * ct2 ) / px2r1 = ( py3r1 + vy3r1 * ct3 ) / px3r1
        -- ct2 = ( ( py3r1 + vy3r1 * ct3 ) / px3r1 * px2r1 - py2r1 ) / vy2r1
        -- ct2 = ( py3r1 * px2r1 / px3r1 / vy2r1 - py2r1 / vy2r1 ) + ( vy3r1 * px2r1 / px3r1 / vy2r1 ) * ct3 

        -- ( pz2r1 + vz2r1 * ct2 ) / px2r1 = ( pz3r1 + vz3r1 * ct3 ) / px3r1
        -- ct3 = ( ( pz2r1 + vz2r1 * ct2 ) / px2r1 * px3r1 - pz3r1 ) / vz3r1
        -- ct3 = ( pz2r1 * px3r1 / px2r1 / vz3r1 - pz3r1 / vz3r1 ) + ( vz2r1 * px3r1 / px2r1 / vz3r1 ) * ct2 

        -- ct3 = ( pz2r1 * px3r1 / px2r1 / vz3r1 - pz3r1 / vz3r1 ) + ( vz2r1 * px3r1 / px2r1 / vz3r1 ) * ( ( py3r1 * px2r1 / px3r1 / vy2r1 - py2r1 / vy2r1 ) + ( vy3r1 * px2r1 / px3r1 / vy2r1 ) * ct3 )
        -- ct3 = ( pz2r1 * px3r1 / px2r1 / vz3r1 - pz3r1 / vz3r1 ) +  ( vz2r1 * px3r1 / px2r1 / vz3r1 ) * ( py3r1 * px2r1 / px3r1 / vy2r1 - py2r1 / vy2r1 ) + ( vz2r1 * px3r1 / px2r1 / vz3r1 ) * ( vy3r1 * px2r1 / px3r1 / vy2r1 ) * ct3
        -- ct3 = ( ( pz2r1 * px3r1 / px2r1 / vz3r1 - pz3r1 / vz3r1 ) + ( vz2r1 * px3r1 / px2r1 / vz3r1 ) * ( py3r1 * px2r1 / px3r1 / vy2r1 - py2r1 / vy2r1 ) ) / ( 1 - vz2r1 * px3r1 * vy3r1 * px2r1 / px2r1 / vz3r1 / px3r1 / vy2r1 )

        SELECT
            ( ( p2r1[3] / p2r1[1] / p3r1[6] * p3r1[1] - p3r1[3] / p3r1[6] ) + ( p2r1[6] / p2r1[1] / p3r1[6] * p3r1[1] ) * ( p3r1[2] / p3r1[1] / p2r1[5] * p2r1[1] - p2r1[2] / p2r1[5] ) ) / ( 1 - p2r1[6] / p2r1[1] / p3r1[6] / p3r1[1] / p2r1[5] * p3r1[1] * p3r1[5] * p2r1[1] ) AS ct3,
            ( p3r1[2] / p3r1[1] / p2r1[5] * p2r1[1] - p2r1[2] / p2r1[5] ) + ( p3r1[5] / p3r1[1] / p2r1[5] * p2r1[1] ) * ct3 AS ct2,
            p2r1[1] AS cpx2r1,
            p2r1[2] + p2r1[5] * ct2 AS cpy2r1,
            p2r1[3] + p2r1[6] * ct2 AS cpz2r1,
            p3r1[1] AS cpx3r1,
            p3r1[2] + p3r1[5] * ct3 AS cpy3r1,
            p3r1[3] + p3r1[6] * ct3 AS cpz3r1,
            cpx2r1 + p1[1] AS cpx2,
            cpy2r1 + p1[2] AS cpy2,
            cpz2r1 + p1[3] AS cpz2,
            cpx3r1 + p1[1] AS cpx3,
            cpy3r1 + p1[2] AS cpy3,
            cpz3r1 + p1[3] AS cpz3,
            ( cpx3 - cpx2 ) / ( ct3 - ct2 ) AS vx,
            ( cpy3 - cpy2 ) / ( ct3 - ct2 ) AS vy,
            ( cpz3 - cpz2 ) / ( ct3 - ct2 ) AS vz,
            cpx3 - vx * ct3 AS px,
            cpy3 - vy * ct3 AS py,
            cpz3 - vz * ct3 AS pz
        FROM relative_axis
    )
    SELECT CAST(px + py + pz AS BIGINT) ans
    FROM cross_all
""" % input)

┌─────────────────┐
│       ans       │
│      int64      │
├─────────────────┤
│ 527310134398221 │
└─────────────────┘